In [2]:
import pandas as pd
import numpy as np
from surprise import SVD, accuracy
from surprise import Reader, Dataset

In [9]:
rating = pd.read_csv("data/mldataset_202301121706.csv", encoding='utf-8')
rating = pd.DataFrame(rating, columns=["u_id","r_fetv_name","r_fetv_est"])

rating.head()

,u_id,r_fetv_name,r_fetv_est
0,test,2022 동아시아 주류문화 페스티벌,4.5
1,test,2022 D.FESTA 대학로거리공연축제,4.5
2,test,제15회 의정부 부대찌개 축제,4.5
3,test,2022 정조대왕 능행차 공동재현,4.5
4,one1,유성정월대보름제,4.5


In [10]:
rating['u_id'].value_counts()
rating['r_fetv_name'].value_counts()

# 평가한 적 없는 축제 가려냄
tab=pd.crosstab(rating['u_id'], rating['r_fetv_name'])
tab

r_fetv_name,2022 D.FESTA 대학로거리공연축제,2022 ZER01NE DAY,2022 대구 떡볶이 페스티벌,2022 동아시아 주류문화 페스티벌,2022 보드게임콘,2022 정조대왕 능행차 공동재현,2022 제50회 우륵문화제,강화도왕방마을겨울축제,국립민속박물관 정월대보름 한마당,동강국제사진제,...,아시아프,아침고요수목원 무궁화전시회,원주옥상영화제,유성정월대보름제,전남국제수묵비엔날레,전주 단오,제15회 의정부 부대찌개 축제,지리산천왕축제,태화강국제설치미술제,한림공원 매화축제
u_id,,,,,,,,,,,,,,,,,,,,,
jungin,0,1,0,0,0,0,1,0,0,1,...,0,0,1,0,0,1,0,1,1,0
one1,0,0,0,0,0,0,0,1,0,0,...,1,0,0,1,1,0,0,0,0,1
test,1,0,0,1,0,1,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
two2,0,0,1,0,1,0,0,0,1,0,...,0,1,0,0,0,0,0,0,0,0


In [11]:
# 두 개의 집단변수를 가지고 나머지 rating을 그룹화
rating_g = rating.groupby(['u_id', 'r_fetv_name'])
rating_g.sum()
tab = rating_g.sum().unstack() # 행렬구조로 변환
tab

r_fetv_est                                    \
r_fetv_name 2022 D.FESTA 대학로거리공연축제 2022 ZER01NE DAY 2022 대구 떡볶이 페스티벌   
u_id                                                                   
jungin                         NaN              4.5              NaN   
one1                           NaN              NaN              NaN   
test                           4.5              NaN              NaN   
two2                           NaN              NaN              4.5   

                                                                               \
r_fetv_name 2022 동아시아 주류문화 페스티벌 2022 보드게임콘 2022 정조대왕 능행차 공동재현 2022 제50회 우륵문화제   
u_id                                                                            
jungin                      NaN        NaN                NaN             4.5   
one1                        NaN        NaN                NaN             NaN   
test                        4.5        NaN                4.5             NaN   
two2                        NaN        4.5                NaN             NaN   

                                                   ...                      \
r_fetv_name 강화도왕방마을겨울축제 국립민속박물관 정월대보름 한마당 동강국제사진제  ... 아시아프 아침고요수목원 무궁화전시회   
u_id                                               ...                       
jungin              NaN               NaN     4.5  ...  NaN            NaN   
one1                4.5               NaN     NaN  ...  4.5            NaN   
test                NaN               NaN     NaN  ...  NaN            NaN   
two2                NaN               4.5     NaN  ...  NaN            4.5   

                                                                        \
r_fetv_name 원주옥상영화제 유성정월대보름제 전남국제수묵비엔날레 전주 단오 제15회 의정부 부대찌개 축제 지리산천왕축제   
u_id                                                                     
jungin          4.5      NaN        NaN   4.5              NaN     4.5   
one1            NaN      4.5        4.5   NaN              NaN     NaN   
test            NaN      NaN        NaN   NaN              4.5     NaN   
two2            NaN      NaN        NaN   NaN              NaN     NaN   

                                  
r_fetv_name 태화강국제설치미술제 한림공원 매화축제  
u_id                              
jungin             4.5       NaN  
one1               NaN       4.5  
test               NaN       NaN  
two2               NaN       NaN  

[4 rows x 22 columns]

In [50]:
temp = tab[tab.isna()].iloc[0] # type: ignore
temp = temp.index.to_list()
res=[]
for t in temp :
    res.append(t[-1])
res

['2022 D.FESTA 대학로거리공연축제',
 '2022 ZER01NE DAY',
 '2022 대구 떡볶이 페스티벌',
 '2022 동아시아 주류문화 페스티벌',
 '2022 보드게임콘',
 '2022 정조대왕 능행차 공동재현',
 '2022 제50회 우륵문화제',
 '강화도왕방마을겨울축제',
 '국립민속박물관 정월대보름 한마당',
 '동강국제사진제',
 '뮤직플로우페스티벌(Music Flow Festival)',
 '봉화송이축제',
 '아시아프',
 '아침고요수목원 무궁화전시회',
 '원주옥상영화제',
 '유성정월대보름제',
 '전남국제수묵비엔날레',
 '전주 단오',
 '제15회 의정부 부대찌개 축제',
 '지리산천왕축제',
 '태화강국제설치미술제',
 '한림공원 매화축제']

In [51]:
#rating 데이터셋 생성
reader = Reader(rating_scale=(1,5))
data = Dataset.load_from_df(df=rating, reader=reader)

data

In [13]:
train = data.build_full_trainset() # 훈련셋
test = train.build_testset() # 검정셋

In [14]:
help(SVD)
# 모델 생성
model = SVD(n_factors=100, n_epochs=20, random_state=123)
model.fit(train) # model 생성

Help on class SVD in module surprise.prediction_algorithms.matrix_factorization:

class SVD(surprise.prediction_algorithms.algo_base.AlgoBase)
 |  SVD(n_factors=100, n_epochs=20, biased=True, init_mean=0, init_std_dev=0.1, lr_all=0.005, reg_all=0.02, lr_bu=None, lr_bi=None, lr_pu=None, lr_qi=None, reg_bu=None, reg_bi=None, reg_pu=None, reg_qi=None, random_state=None, verbose=False)
 |  
 |  The famous *SVD* algorithm, as popularized by `Simon Funk
 |  <https://sifter.org/~simon/journal/20061211.html>`_ during the Netflix
 |  Prize. When baselines are not used, this is equivalent to Probabilistic
 |  Matrix Factorization :cite:`salakhutdinov2008a` (see :ref:`note
 |  <unbiased_note>` below).
 |  
 |  The prediction :math:`\hat{r}_{ui}` is set as:
 |  
 |  .. math::
 |      \hat{r}_{ui} = \mu + b_u + b_i + q_i^Tp_u
 |  
 |  If user :math:`u` is unknown, then the bias :math:`b_u` and the factors
 |  :math:`p_u` are assumed to be zero. The same applies for item :math:`i`
 |  with :math:`b_

In [52]:
# jungin 사용자에게 축제 추천
user_id = 'jungin' # 추천대상자
item_ids = res # 추천 대상 축제 리스트
actual_rating = 0 # 실제 평점

result = []
for item_id in item_ids :
    predict = model.predict(user_id, item_id, actual_rating)
    #print(predict)
    
    #pred = np.sort(predict, axis=0)
    #print(pred)
    temp = [predict.uid, predict.iid, predict.est]
    result.append(temp)
result.sort(key=lambda x:(x[0], x[1]), reverse=True)

num = 1
for r in result[:3] :
    print('Top',num,'::',r[1])
    num+=1

Top 1 :: 한림공원 매화축제
Top 2 :: 태화강국제설치미술제
Top 3 :: 지리산천왕축제
